# Unsaturated poroelasticity: Convergence Test #2

* Author: Jhabriel Varela
* E-mail: jhabriel.varela@uib.no
* Date: 10.04.2019
* Institution: PMG - UiB - Norway

## Description of the problem

Unlike the previous convergence test (see [[1](#ref)]), in this notebook we employ a more realistic set of constitutive relationships, i.e.: the van Genuchten-Mualem retention curves [[2](#ref)]. Moreover, we include gravity contributions that were previously neglected. 

Since we are not only interested in the flow response but in the mechanical effects as well, instead of using the pressure head $\psi$, the water content $\theta$ and the specific moisture capacity $C^{\theta}$, we explictly describe the dependency of the the saturation $S$, specific saturation capacity $C^S$ and relative permeability $k_r^w$ in terms of the pressure $p$. Note that there is a straightforward relation among these quantities:

$$
\psi = \frac{p}{\gamma}, \qquad \theta = nS, \qquad C^{\theta} = \frac{n}{\gamma} C^S,
$$
where $n$ is the porosity and $\gamma$ is the specific gravity.

The (slightly modified) van Genuchten-Mualem relations hold:

$$
S(p) = \begin{cases} \displaystyle{\frac{1 - S_r}{\left[1 + \left(\frac{\alpha_v}{\gamma} |p|\right)^{n_v} \right]^{m_v}} + S_r} & p < 0 \\
S_r & p \geq 0 \end{cases},
$$

$$
C^S(p) = \frac{\partial S}{\partial p} = \begin{cases} \displaystyle{\frac{-m_v n_v (1-S_r)p\left(\frac{\alpha_v}{\gamma} |p|\right)^{n_v}}{|p|^2 \left[1 + \left(\frac{\alpha_v}{\gamma} |p|\right)^{n_v} \right]^{m_v+1}}} & p < 0 \\
0 & p \geq 0 \end{cases},
$$

$$
k_r^w (p) = \begin{cases} \displaystyle{\frac{\Big\lbrace 1 - \left(\frac{\alpha_v}{\gamma} |p|\right)^{n_v-1} \left[1 + \left(\frac{\alpha_v}{\gamma} |p|\right)^{n_v}\right]^{n_v}\Big\rbrace^2}{\left[1+\left(\frac{\alpha_v}{\gamma} |p|\right)^{n_v} \right]^{m_v/2}}} & p < 0 \\
1 & p \geq 0 \end{cases},
$$

where $\alpha_v$, $n_v$ and $m_v$ are equation parameters and $S_r$ is the residual saturation.

In this problem we use $\alpha_v = 6$, $n_v = 1.5$, $m_v = 0.3333$ and $S_r = 0.25$. Moreover, we set $\lambda_s = \mu_s = K = C_s = \rho_f = \mu_f = C_f = g = \alpha = 1$ and $n = 0.5$. 

The domain, boundary conditions, initial conditions and analytical solutions are the same as those used in [[1](#ref)]. We measure the errors using the same norms defined in [[1](#ref)]. 

The final simulation time is 1 and we study the convergence using the following pairs of spatial $h$ and time steps $\tau$: $(0.1,0.1)$, $(0.05,0.05)$, $(0.025,0.025)$ and $(0.0125,0.0125)$.

As expected, we get a second order convergence on the primary variables $p$ and $\mathbf{u}$.

## Importing modules

In [4]:
import numpy as np
import scipy.sparse as sps
import matplotlib.pyplot as plt
import scipy.optimize as opt
import porepy as pp
from porepy.ad.forward_mode import Ad_array
np.set_printoptions(precision=4, suppress = True)

## Convergence analysis function

In [5]:
def convergence_analysis(cells_num,time_levels):    
    
    ### Mechanics source term

    def source_mech(g,t):

        # x and  y cell centers
        x = g.cell_centers[0]
        y = g.cell_centers[1]

        # Initializing mechanics source term
        F = np.zeros(g.dim * g.num_cells)

        F[::2]  =  (3/(4*((6*np.abs(t*x*y*(x - 1)*(y - 1) + 1))**(3/2) + 1)**(1/3)) + 1/4)*(t*x*y*(y - 1) + t*y*(x - 1)*(y - 1)) + 2*t*x*(x - 1) - 2*t*x*(y - 1) - 2*t*y*(x - 1) + 6*t*y*(y - 1) - 2*t*(x - 1)*(y - 1) - 2*t*x*y - (9*np.sign(t*x*y*(x - 1)*(y - 1) + 1)*(6*np.abs(t*x*y*(x - 1)*(y - 1) + 1))**(1/2)*(t*x*y*(x - 1)*(y - 1) + 1)*(t*x*y*(y - 1) + t*y*(x - 1)*(y - 1)))/(4*((6*np.abs(t*x*y*(x - 1)*(y - 1) + 1))**(3/2) + 1)**(4/3))
        F[1::2] =  (3/(4*((6*np.abs(t*x*y*(x - 1)*(y - 1) + 1))**(3/2) + 1)**(1/3)) + 1/4)*(t*x*y*(x - 1) + t*x*(x - 1)*(y - 1)) - 6*t*x*(x - 1) + 2*t*x*(y - 1) + 2*t*y*(x - 1) - 2*t*y*(y - 1) + 2*t*(x - 1)*(y - 1) + 2*t*x*y - (9*np.sign(t*x*y*(x - 1)*(y - 1) + 1)*(6*np.abs(t*x*y*(x - 1)*(y - 1) + 1))**(1/2)*(t*x*y*(x - 1)*(y - 1) + 1)*(t*x*y*(x - 1) + t*x*(x - 1)*(y - 1)))/(4*((6*np.abs(t*x*y*(x - 1)*(y - 1) + 1))**(3/2) + 1)**(4/3))

        return F

    ### Flow source term

    def source_flow(g,t):

        # x and  y cell centers
        x = g.cell_centers[0]
        y = g.cell_centers[1]

        # Flow source term
        f = (2*((3*np.sign(t*x*y*(x - 1)*(y - 1) + 1)*(t*x*y*(x - 1) + t*x*(x - 1)*(y - 1)))/(((6*np.abs(t*x*y*(x - 1)*(y - 1) + 1))**(3/2) + 1)**(1/3)*(6*np.abs(t*x*y*(x - 1)*(y - 1) + 1))**(1/2)) - (18*np.abs(t*x*y*(x - 1)*(y - 1) + 1)*np.sign(t*x*y*(x - 1)*(y - 1) + 1)*(t*x*y*(x - 1) + t*x*(x - 1)*(y - 1)))/((6*np.abs(t*x*y*(x - 1)*(y - 1) + 1))**(3/2) + 1)**(4/3))*((6*np.abs(t*x*y*(x - 1)*(y - 1) + 1))**(1/2)/((6*np.abs(t*x*y*(x - 1)*(y - 1) + 1))**(3/2) + 1)**(1/3) - 1)*(t*x*y*(x - 1) + t*x*(x - 1)*(y - 1) - 1))/((6*np.abs(t*x*y*(x - 1)*(y - 1) + 1))**(3/2) + 1)**(1/6) - (3/(4*((6*np.abs(t*x*y*(x - 1)*(y - 1) + 1))**(3/2) + 1)**(1/3)) + 1/4)*(x*y*(x - 1) - x*y*(y - 1) + x*(x - 1)*(y - 1) - y*(x - 1)*(y - 1)) + (2*((3*np.sign(t*x*y*(x - 1)*(y - 1) + 1)*(t*x*y*(y - 1) + t*y*(x - 1)*(y - 1)))/(((6*np.abs(t*x*y*(x - 1)*(y - 1) + 1))**(3/2) + 1)**(1/3)*(6*np.abs(t*x*y*(x - 1)*(y - 1) + 1))**(1/2)) - (18*np.abs(t*x*y*(x - 1)*(y - 1) + 1)*np.sign(t*x*y*(x - 1)*(y - 1) + 1)*(t*x*y*(y - 1) + t*y*(x - 1)*(y - 1)))/((6*np.abs(t*x*y*(x - 1)*(y - 1) + 1))**(3/2) + 1)**(4/3))*((6*np.abs(t*x*y*(x - 1)*(y - 1) + 1))**(1/2)/((6*np.abs(t*x*y*(x - 1)*(y - 1) + 1))**(3/2) + 1)**(1/3) - 1)*(t*x*y*(y - 1) + t*y*(x - 1)*(y - 1)))/((6*np.abs(t*x*y*(x - 1)*(y - 1) + 1))**(3/2) + 1)**(1/6) - (3*np.sign(t*x*y*(x - 1)*(y - 1) + 1)*((6*np.abs(t*x*y*(x - 1)*(y - 1) + 1))**(1/2)/((6*np.abs(t*x*y*(x - 1)*(y - 1) + 1))**(3/2) + 1)**(1/3) - 1)**2*(6*np.abs(t*x*y*(x - 1)*(y - 1) + 1))**(1/2)*(t*x*y*(y - 1) + t*y*(x - 1)*(y - 1))**2)/(2*((6*np.abs(t*x*y*(x - 1)*(y - 1) + 1))**(3/2) + 1)**(7/6)) + (2*t*x*((6*np.abs(t*x*y*(x - 1)*(y - 1) + 1))**(1/2)/((6*np.abs(t*x*y*(x - 1)*(y - 1) + 1))**(3/2) + 1)**(1/3) - 1)**2*(x - 1))/((6*np.abs(t*x*y*(x - 1)*(y - 1) + 1))**(3/2) + 1)**(1/6) + (2*t*y*((6*np.abs(t*x*y*(x - 1)*(y - 1) + 1))**(1/2)/((6*np.abs(t*x*y*(x - 1)*(y - 1) + 1))**(3/2) + 1)**(1/3) - 1)**2*(y - 1))/((6*np.abs(t*x*y*(x - 1)*(y - 1) + 1))**(3/2) + 1)**(1/6) - x*y*(x - 1)*(y - 1)*(3/(8*((6*np.abs(t*x*y*(x - 1)*(y - 1) + 1))**(3/2) + 1)**(1/3)) + (3/(4*((6*np.abs(t*x*y*(x - 1)*(y - 1) + 1))**(3/2) + 1)**(1/3)) + 1/4)**2/2 + 1/8) - (3*np.sign(t*x*y*(x - 1)*(y - 1) + 1)*((6*np.abs(t*x*y*(x - 1)*(y - 1) + 1))**(1/2)/((6*np.abs(t*x*y*(x - 1)*(y - 1) + 1))**(3/2) + 1)**(1/3) - 1)**2*(6*np.abs(t*x*y*(x - 1)*(y - 1) + 1))**(1/2)*(t*x*y*(x - 1) + t*x*(x - 1)*(y - 1))*(t*x*y*(x - 1) + t*x*(x - 1)*(y - 1) - 1))/(2*((6*np.abs(t*x*y*(x - 1)*(y - 1) + 1))**(3/2) + 1)**(7/6)) + (9*x*y*np.sign(t*x*y*(x - 1)*(y - 1) + 1)*((3/(8*((6*np.abs(t*x*y*(x - 1)*(y - 1) + 1))**(3/2) + 1)**(1/3)) + 1/8)*(t*x*y*(x - 1)*(y - 1) + 1) - 1/2)*(6*np.abs(t*x*y*(x - 1)*(y - 1) + 1))**(1/2)*(x - 1)*(y - 1))/(4*((6*np.abs(t*x*y*(x - 1)*(y - 1) + 1))**(3/2) + 1)**(4/3))
        f = np.array(f,dtype=float)

        return f

    ### Analytical solution

    def analytical(g,t):

        x_cntr = g.cell_centers[0]
        y_cntr = g.cell_centers[1]

        u = np.zeros(g.dim * g.num_cells)

        u[::2] = t * (1-x_cntr) * x_cntr * (1-y_cntr) * y_cntr
        u[1::2] = - u[::2]

        p = -t * (1-x_cntr) * x_cntr * (1-y_cntr) * y_cntr - 1

        return u,p

    ### Computation of the arithmetic averaged relative permeabilities

    def arithmetic_average(krw,g,bc,bc_val,p_m):

        """
        Computes the arithmetic average of the relative permability

        SYNOPSIS:
            arithmetic_average(krw,g,bc,bc_val,p_m)

        INPUT ARGUMENTS:
            krw              - Lambda function, relative permeability function krw = f(psi)
            g                - PorePy grid object
            bc               - PorePy scalar boundary object
            bc_val           - NumPy array, containing values of boundary conditions
            p_m              - NumPy array, containing values of pressure at the cell centers

        RETURNS:
            krw_ar           - Numpy array, contatining arithmetic averaged relative permeabilities 
                               at the face centers
        """

        neu_fcs = bc.is_neu.nonzero()    # neumann boundary faces
        dir_fcs = bc.is_dir.nonzero()    # dirichlet boundary faces
        int_fcs = g.get_internal_faces() # internal faces

        fcs_neigh = np.zeros((g.num_faces,2),dtype=int) #          
        fcs_neigh[:,0] = g.cell_face_as_dense()[0]      # faces neighbouring mapping
        fcs_neigh[:,1] = g.cell_face_as_dense()[1]      # 

        int_fcs_neigh = fcs_neigh[int_fcs]              # internal faces neighbouring mapping

        # Initializing 
        krw_ar = np.zeros(g.num_faces)

        # Neumann boundaries relative permeabilities
        krw_ar[neu_fcs] = 1.

        # Dirichlet boundaries relative permeabilities
        dir_cells_neigh = fcs_neigh[dir_fcs] # neighboring cells of dirichlet faces
        dir_cells = dir_cells_neigh[(dir_cells_neigh >= 0).nonzero()] # cells that share a dirichlet face 
        krw_ar[dir_fcs] = 0.5 * (krw(bc_val[dir_fcs]) + 
                                 krw(p_m[dir_cells]))

        # Internal faces relative permeabilities
        krw_ar[int_fcs] = 0.5 * (krw(p_m[int_fcs_neigh[:,0]]) +
                                 krw(p_m[int_fcs_neigh[:,1]]))

        return krw_ar

    ## Setting-up the grid

    Nx = Ny = cells_num
    Lx = Ly = 1
    g = pp.CartGrid([Nx,Ny], [Lx,Ly])
    g.compute_geometry()
    V = g.cell_volumes
    y_cntr = g.cell_centers[1]
    y_face = g.face_centers[1]
    d = dict() # initializing data dictionary

    ## Physical parameters

    ### Elasticity

    mu_s = 1          # [Pa] First Lame parameter
    lambda_s = 1     # [Pa] Second Lame parameter

    ### Rock

    K = 1           # [m^2] Intrinsic permeability
    n = 0.5             # [-] Porosity
    C_s = 1         # [1/Pa] Compressibility

    ### Fluid

    mu_f = 1            # [Pa.s] Viscosity
    rho_f = 1           # [kg/m^3] Density
    grav = 1            # [m/s^2] Gravity
    gamma = rho_f*grav  # [Pa/m] Specific gravity
    C_f = 1             # [1/Pa] Compressibility

    ### Medium          

    alpha = 1           # [-] Biot's coefficient

    ### van Genuchten

    alpha_v = 6         # [1/m] van Genuchten parameter
    n_v = 1.5           # [-] van Genuchten parameter
    m_v = 1-(1/n_v)     # [-] van Genuchten parameter
    S_r = 0.25          # [-] Residual saturation
    a_v = alpha_v/gamma # [1/Pa] van Genuchten (artificial) parameter

    ## Creating the second and fourth order tensors

    perm = pp.SecondOrderTensor(g.dim, K*np.ones(g.num_cells))
    constit = pp.FourthOrderTensor(g.dim, mu_s*np.ones(g.num_cells), lambda_s*np.ones(g.num_cells))

    ## Boundary conditions

    b_faces = g.tags['domain_boundary_faces'].nonzero()[0]

    # Extracting indices of boundary faces w.r.t g
    x_min = b_faces[g.face_centers[0,b_faces] < 0.0001]
    x_max = b_faces[g.face_centers[0,b_faces] > 0.9999*Lx]
    y_min = b_faces[g.face_centers[1,b_faces] < 0.0001]
    y_max = b_faces[g.face_centers[1,b_faces] > 0.9999*Ly]

    # Extracting indices of boundary faces w.r.t b_faces
    west   = np.in1d(b_faces,x_min).nonzero()
    east   = np.in1d(b_faces,x_max).nonzero()
    south  = np.in1d(b_faces,y_min).nonzero()
    north  = np.in1d(b_faces,y_max).nonzero()

    ### Flow Boundary conditions

    # Setting the tags at each boundary side
    labels_flow = np.array([None]*b_faces.size)
    labels_flow[east]   = 'dir'
    labels_flow[west]   = 'dir'
    labels_flow[south]  = 'dir'
    labels_flow[north]  = 'dir'

    # Constructing the bc object
    bc_flow = pp.BoundaryCondition(g, b_faces, labels_flow)

    # Gravity contribution to the boundary conditions
    bc_val_grav = np.zeros(g.num_faces)
    bc_val_grav[x_min] = gamma * y_face[x_min]
    bc_val_grav[x_max] = gamma * y_face[x_max]
    bc_val_grav[y_min] = gamma * y_face[y_min]
    bc_val_grav[y_max] = gamma * y_face[y_max]

    # Pressure contribution to the boundary conditions 
    bc_val_pres = np.zeros(g.num_faces)
    bc_val_pres[x_min] = -1
    bc_val_pres[x_max] = -1
    bc_val_pres[y_min] = -1
    bc_val_pres[y_max] = -1

    # Construction the boundary values array
    bc_val_flow = bc_val_pres

    ### Elasticity Boundary conditions

    # Setting the tags at each boundary side
    labels_mech = np.array([None]*b_faces.size)
    labels_mech[east]   = 'dir'
    labels_mech[west]   = 'dir'
    labels_mech[south]  = 'dir'
    labels_mech[north]  = 'dir'

    # Constructing the bc object
    bc_mech = pp.BoundaryConditionVectorial(g, b_faces, labels_mech)

    # Constructing the boundary values array
    bc_val_mech = np.zeros(g.num_faces * g.dim)

    ## Creating data objects

    ### Mechanics data

    specified_parameters_mech = {"fourth_order_tensor": constit, 
                                 "bc": bc_mech, 
                                 "biot_alpha" : 1.,
                                 "bc_values": bc_val_mech}

    d = pp.initialize_default_data(g,d,"mechanics", specified_parameters_mech)

    ### Flow data

    specified_parameters_flow = {"second_order_tensor": perm, 
                                 "bc": bc_flow, 
                                 "biot_alpha": 1.,
                                 "bc_values": bc_val_flow}

    d = pp.initialize_default_data(g,d,"flow", specified_parameters_flow)

    ## Performing MPSA/MPFA discretization

    ### Discretization

    solver_biot = pp.Biot("mechanics","flow")
    solver_biot.discretize(g,d)

    ### Retrieving operators

    biot_F = d['discretization_matrices']['flow']['flux']
    biot_boundF = d['discretization_matrices']['flow']['bound_flux']
    biot_compat = d['discretization_matrices']['flow']['biot_stabilization']
    biot_divF = pp.fvutils.scalar_divergence(g)

    biot_S = d['discretization_matrices']['mechanics']['stress']
    biot_boundS = d['discretization_matrices']['mechanics']['bound_stress']
    biot_divU = d['discretization_matrices']['mechanics']['div_d']
    biot_gradP = d['discretization_matrices']['mechanics']['grad_p']
    biot_boundUCell = d['discretization_matrices']['mechanics']['bound_displacement_cell']
    biot_boundUFace = d['discretization_matrices']['mechanics']['bound_displacement_face']
    biot_boundUPressure = d['discretization_matrices']['mechanics']['bound_displacement_pressure']
    biot_divS = pp.fvutils.vector_divergence(g)

    ### Creating discrete operators

    F      = lambda x: biot_F * x                       #
    boundF = lambda x: biot_boundF * x                  # Flow 
    compat = lambda x: biot_compat * x                  # operators
    divF    = lambda x: biot_divF * x                   #

    S      = lambda x: biot_S * x                       #
    boundS = lambda x: biot_boundS * x                  #  
    divU   = lambda x: biot_divU * x                    # 
    divS   = lambda x: biot_divS * x                    # Mechanics 
    gradP  = lambda x: biot_divS * biot_gradP * x       # operators
    boundUCell = lambda x: biot_boundUCell * x          #
    boundUFace = lambda x : biot_boundUFace * x         #
    boundUPressure = lambda x: biot_boundUPressure * x  # 

    ## Creating AD variables

    u_init = np.zeros(g.dim * g.num_cells)    # initial displacement field
    p_init = -1*np.ones(g.num_cells)          # initial pressure distribution

    u_ad = Ad_array(u_init.copy(), sps.diags(np.ones(g.num_cells * g.dim))) # initializing u_ad
    p_ad = Ad_array(p_init.copy(), sps.diags(np.ones(g.num_cells)))         # initializing p_ad

    ## Water retention curves

    # Boolean lambda function that determines if we are in the unsat (true) or sat (false) zone
    is_unsat = lambda p: p < 0

    # Water saturation
    Sw = lambda p: is_unsat(p) * (((1-S_r)/(1+(a_v * np.abs(p))**n_v)**m_v) + S_r) \
                   + (1-is_unsat(p)) * S_r

    # Specific saturation capacity
    C = lambda p:  is_unsat(p) * ((-m_v*n_v*(1-S_r)*p*(a_v*np.abs(p))**n_v)/(np.abs(p)**2*((a_v*np.abs(p))**n_v+1)**(m_v+1))) \
                   + (1-is_unsat(p)) * 0

    # Relative permeability
    krw = lambda p: is_unsat(p) * ((1 - (a_v*np.abs(p))**(n_v-1)*(1+(a_v*np.abs(p))**n_v)**(-m_v))**2 / (1+(a_v*np.abs(p))**n_v)**(m_v/2)) \
                    + (1-is_unsat(p)) * 1

    ## Time parameters

    t0 = 0                                # [s] Initial time
    tf = 1                                # [s] Final simulation time
    tLevels = time_levels                 # [-] Time levels
    times = np.linspace(t0,tf,tLevels+1)  # [s] Vector of time evaluations
    dt = np.diff(times)                   # [s] Vector of time steps

    ## Discrete equations

    # Arithmetic mean of relative permeability
    krw_ar = lambda p_m: arithmetic_average(krw,g,bc_flow,bc_val_flow,p_m)

    # Generalized Hooke's law
    T = lambda u: S(u) + boundS(bc_val_mech)

    # Momentum conservation equation (Mechanics contribution)
    u_eq1 = lambda u: divS(T(u)) 

    # Momentum conservation equation  (Flow contribution)
    u_eq2 = lambda p,p_n: gradP(p*Sw(p_n)) - source_mech(g,times[tt])*V[0]

    # Multiphase Darcy's law
    Q = lambda p,p_m: (1/mu_f) * (F(p + gamma*y_cntr) + 
                                  boundF(bc_val_flow + bc_val_grav)) * krw_ar(p_m)

    # Auxiliary compressibility-like terms
    chi_p = lambda p_n : (alpha-n)*C_s*Sw(p_n)**2 + n*C_f*Sw(p_n)
    chi_S = lambda p_n : (alpha-n)*C_s*Sw(p_n)*p_n + n

    # Mass conservation equation (Mechanics contribution)
    p_eq1 = lambda u,u_n,p_n,dt: alpha * (divU(u-u_n)/dt) * Sw(p_n)

    # Mass conservation equation (Flow contribution)
    p_eq2 = lambda p,p_m,p_n,dt:  (compat(p-p_n)/dt) * Sw(p_n)  \
                                  + ((p-p_n)/dt) * chi_p(p_n) * V  \
                                  + (((p-p_m)*C(p_m) + Sw(p_m) - Sw(p_n))/dt) * chi_S(p_n) * V \
                                  + divF(Q(p,p_m)) \
                                  - source_flow(g,times[tt]) * V

    ## The time loop

    ## Newton parameters
    newton_param = dict()
    newton_param['max_tol'] = 1E-10            # [cm] maximum absolute tolerance (pressure head)
    newton_param['max_iter'] = 40             # [iter] maximum number of iterations
    newton_param['res_norm'] = 100            # [cm] absolute tolerance
    newton_param['iter'] = 1                  # [iter] iteration
    
    print("\n Performing simulation with h={} and dt={} \n".format(1/cells_num,1/time_levels))

    tt = 0
    while times[tt] < tf:

        u_n = u_ad.val.copy()            # u: current time step
        p_n = p_ad.val.copy();           # p: current time step
        tt += 1

        newton_param.update({'res_norm':1000, 'iter':1})      

        while newton_param['res_norm'] > newton_param['max_tol'] and \
              newton_param['iter'] <= newton_param['max_iter']:

            p_m = p_ad.val.copy()            # current iteration level (m)

            # Calling equations
            eq1 = u_eq1(u_ad)
            eq2 = u_eq2(p_ad,p_n)
            eq3 = p_eq1(u_ad,u_n,p_n,dt[tt-1])
            eq4 = p_eq2(p_ad,p_m,p_n,dt[tt-1])

            # Assembling Jacobian of the coupled system
            J_mech = np.hstack((eq1.jac,eq2.jac)) # Jacobian blocks (mechanics)
            J_flow = np.hstack((eq3.jac,eq4.jac)) # Jacobian blocks (flow)
            J = sps.bmat(np.vstack((J_mech,J_flow)),format='csc') # Jacobian (coupled)

            # Determining residual of the coupled system
            R_mech = eq1.val + eq2.val           # Residual (mechanics)
            R_flow = eq3.val + eq4.val            # Residual (flow)
            R = np.hstack((R_mech,R_flow))        # Residual (coupled)

            y = sps.linalg.spsolve(J,-R)                  # 
            u_ad.val = u_ad.val + y[:g.dim*g.num_cells]   # Newton update
            p_ad.val = p_ad.val + y[g.dim*g.num_cells:]   #

            newton_param['res_norm'] = np.linalg.norm(R)

            if newton_param['res_norm'] <= newton_param['max_tol'] and \
               newton_param['iter'] <= newton_param['max_iter']:
                print('Time: {:.2f} [s], Iter: {} \t Error: {:.2e}'.format(times[tt],newton_param['iter'],newton_param['res_norm']))
            elif newton_param['iter'] > newton_param['max_iter']:
                print('Error: Newton method did not converge!')
            else:
                newton_param['iter'] += 1
                
    ## Analytical solution
    u_ex,p_ex = analytical(g,times[-1])
    
    ## Computing norms
    eps_p = np.linalg.norm(p_ex-p_ad.val)/np.linalg.norm(p_ad.val)
    eps_u = np.linalg.norm(u_ex-u_ad.val)/np.linalg.norm(u_ad.val)
    eps_pu = eps_u + dt[0] * eps_p
    
    return eps_pu

## Performing test

In [6]:
N = T = np.array([10,20,40,80])
h = tau = 1/N

eps_pu = np.zeros(len(N))

for i in range(len(N)):
    eps_pu[i] = convergence_analysis(N[i],T[i])
    
eps_pu_red = np.zeros(len(eps_pu)-1)

for i in range(len(eps_pu_red)):
    eps_pu_red[i] = eps_pu[i]/eps_pu[i+1]


 Performing simulation with h=0.1 and dt=0.1 

Time: 0.10 [s], Iter: 4 	 Error: 5.44e-13
Time: 0.20 [s], Iter: 4 	 Error: 5.50e-13
Time: 0.30 [s], Iter: 4 	 Error: 5.42e-13
Time: 0.40 [s], Iter: 4 	 Error: 5.35e-13
Time: 0.50 [s], Iter: 4 	 Error: 5.30e-13
Time: 0.60 [s], Iter: 4 	 Error: 5.27e-13
Time: 0.70 [s], Iter: 4 	 Error: 5.26e-13
Time: 0.80 [s], Iter: 4 	 Error: 5.26e-13
Time: 0.90 [s], Iter: 4 	 Error: 5.27e-13
Time: 1.00 [s], Iter: 4 	 Error: 5.29e-13

 Performing simulation with h=0.05 and dt=0.05 

Time: 0.05 [s], Iter: 4 	 Error: 8.83e-14
Time: 0.10 [s], Iter: 4 	 Error: 9.22e-14
Time: 0.15 [s], Iter: 4 	 Error: 9.21e-14
Time: 0.20 [s], Iter: 4 	 Error: 9.21e-14
Time: 0.25 [s], Iter: 4 	 Error: 9.21e-14
Time: 0.30 [s], Iter: 4 	 Error: 9.23e-14
Time: 0.35 [s], Iter: 4 	 Error: 9.25e-14
Time: 0.40 [s], Iter: 4 	 Error: 9.29e-14
Time: 0.45 [s], Iter: 4 	 Error: 9.34e-14
Time: 0.50 [s], Iter: 4 	 Error: 9.40e-14
Time: 0.55 [s], Iter: 4 	 Error: 9.47e-14
Time: 0.60 [s], Iter

## Showing results

In [7]:
print('Spatial step size:',h)
print('Time step size:   ',tau)
print('Primary error:    ',eps_pu)
print('Error reduction:  ',np.concatenate(([np.NaN],eps_pu_red)))

Spatial step size: [0.1    0.05   0.025  0.0125]
Time step size:    [0.1    0.05   0.025  0.0125]
Primary error:     [0.0186 0.0047 0.0012 0.0003]
Error reduction:   [   nan 3.9484 4.0036 4.0222]


## References
<a id='ref'></a>

[1]: <url>https://github.com/jhabriel/pp-implementations/blob/master/unsaturated_poroelasticity/convergence_tests/unsat_poro_convergence_test_1.ipynb

[2]: *van Genuchten, M. T. (1980). A closed-form equation for predicting the hydraulic conductivity of unsaturated soils 1. Soil science society of America journal, 44(5), 892-898.*